In [31]:
import sys
module_path = "../"
if module_path not in sys.path:
    sys.path.append(module_path)

In [32]:
from model import *
from data import *
from helpers import *
import numpy as np
import pandas as pd
import yaml
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import CSVLogger
from torch.utils.data import DataLoader, Dataset
import sys
import time

In [33]:
X = torch.Tensor(pd.read_csv('../../data/DataFiles/Algebra30.csv', header=None).values)
Q = torch.Tensor(pd.read_csv('../QMatrices/QMatrixAlgebra.csv', index_col=0).values)


In [34]:
with open("../config.yml", "r") as f:
    cfg = yaml.safe_load(f)
    cfg = cfg['configs']


In [35]:

# initialise model and optimizer
logger = CSVLogger("logs", name='algebra', version=0)
trainer = Trainer(fast_dev_run=cfg['single_epoch_test_run'],
                  max_epochs=cfg['max_epochs'],
                  enable_checkpointing=False, 
                  logger=False, 
                  callbacks=[EarlyStopping(monitor='train_loss', min_delta=cfg['min_delta'], patience=cfg['patience'], mode='min')])

dataset = SimDataset(X)
train_loader = DataLoader(dataset, batch_size=cfg['batch_size'], shuffle=False)
vae = CVAE(nitems=X.shape[1],
            dataloader=train_loader,
            latent_dims=cfg['mirt_dim'],
            hidden_layer_size=cfg['hidden_layer_size'],
            qm=Q,
            learning_rate=cfg['learning_rate'],
            batch_size=cfg['batch_size'],
            beta=cfg['beta'],
            n_samples=cfg['n_iw_samples']
          )

start = time.time()
trainer.fit(vae)
runtime = time.time()-start
print(runtime)

a_est = vae.decoder.weights.t().detach().numpy()
d_est = vae.decoder.bias.t().detach().numpy()

dataset = SimDataset(X)
train_loader = DataLoader(dataset, batch_size=X.shape[0], shuffle=False)
data, mask = next(iter(train_loader))
_, log_sigma_est  = vae.encoder(data, mask)
post_samples = vae.fscores((data, mask), cfg['model'])
theta_est = post_samples.mean(0)


for d in range(9):
    if np.corrcoef(theta_est[:,d], theta_est_full[:,d])[1,0] < 0:
        a_est[:,d] *= -1
        theta_est[:,d] *=-1
    


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/Users/karel/Documents/GitHub/VAE-MIRT-Missing/MIRTVAE/additional_scripts/../data.py:46: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.x_train = torch.tensor(X, dtype=torch.float32)

  | Name      | Type               | Params
-------------------------------------------------
0 | encoder   | ConditionalEncoder | 3.4 K 
1 | sampler   | SamplingLayer      | 0     
2 | transform | Identity           | 0     
3 | decoder   | Decoder            | 750   
-------------------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)
/Library/Frameworks/Python.framework/Versions/3.10/lib/pyth

0
Epoch 62: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 71.60it/s]
36.40827298164368


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params
-------------------------------------------------
0 | encoder   | ConditionalEncoder | 3.4 K 
1 | sampler   | SamplingLayer      | 0     
2 | transform | Identity           | 0     
3 | decoder   | Decoder            | 750   
-------------------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)


1
Epoch 48:  20%|████████████████████████████████▊                                                                                                                                       | 8/41 [00:00<00:00, 71.57it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



...

Epoch 53: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 66.75it/s]
32.676167011260986


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name      | Type               | Params
-------------------------------------------------
0 | encoder   | ConditionalEncoder | 3.4 K 
1 | sampler   | SamplingLayer      | 0     
2 | transform | Identity           | 0     
3 | decoder   | Decoder            | 750   
-------------------------------------------------
4.2 K     Trainable params
0         Non-trainable params
4.2 K     Total params
0.017     Total estimated model params size (MB)


14
Epoch 39:  63%|█████████████████████████████████████████████████████████████████████████████████████████████████████████▉                                                             | 26/41 [00:00<00:00, 69.59it/s]

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Epoch 53: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 41/41 [00:00<00:00, 69.99it/s]
32.984330892562866


In [40]:
mu = theta_est.repeat(500, 1, 1)
sigma = log_sigma_est.repeat(500, 1, 1)
vae.transform.n_samples = 500
z = vae.sampler(mu, sigma)
z_transformed = vae.transform(z)



# For each of the 10000 elements, we need to compute the corss products
cov_matrix = torch.einsum('ijk,ijl->jkl', z_transformed, z_transformed) / (z_transformed.shape[0] - 1)

# Step 4: Convert the covariance matrix to a correlation matrix
std_dev = torch.sqrt(torch.diagonal(cov_matrix, dim1=-2, dim2=-1)).unsqueeze(-1)
est_cov_mat = (cov_matrix / (std_dev @ std_dev.transpose(-2, -1))).detach().numpy().mean(0)
est_cor_mat = np.abs(cov2cor(est_cov_mat))
est_cors = np.corrcoef(theta_est.T)[np.triu_indices(9, 1)]

TypeError: repeat() takes at most 2 arguments (3 given)

In [41]:
est_cors = np.corrcoef(theta_est.T)

In [42]:
np.savetxt('/Users/karel/Documents/GitHub/VAE-MIRT-Missing/data/algebra/a_est', a_est, 
              delimiter = ",")
np.savetxt('/Users/karel/Documents/GitHub/VAE-MIRT-Missing/data/algebra/d_est', d_est, 
              delimiter = ",")
np.savetxt('/Users/karel/Documents/GitHub/VAE-MIRT-Missing/data/algebra/theta_est', theta_est, 
              delimiter = ",")
np.savetxt('/Users/karel/Documents/GitHub/VAE-MIRT-Missing/data/algebra/cor_est', est_cors, 
              delimiter = ",")


In [ ]:
import pandas as pd
a_est_full = pd.read_csv('/Users/karel/Documents/GitHub/VAE-MIRT-Missing/data/algebra/a_est_full', index_col=0).values
theta_est_full = pd.read_csv('/Users/karel/Documents/GitHub/VAE-MIRT-Missing/data/algebra/theta_est_full', index_col=0).values
